In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

## Preproceso de datos y Cálculo de la Polaridad

In [2]:
tweets = pd.read_csv('nflx_all_tweets.csv', index_col=0)

In [3]:
tweets.head()

,date,text
35982,2017-11-23,Scandal creator Why I left ABC for Netflix DIS NFLX
35957,2017-11-23,Netflix Inc NFLX Stock Live Analysis
35956,2017-11-23,Netflix NFLX Market Valuation Rose While Parkwood Trimmed by Million Its Holding
35955,2017-11-23,Strong Price Compression from a Daily Rising Wedge red NFLX
35954,2017-11-23,VMware Analysts think these stocks can be a game changer Netflix Inc NASDAQ NFLX Retail via TheCloudNetwork on


In [4]:
tweets['date'] = pd.to_datetime(tweets.date)

In [5]:
import nltk
from nltk.tokenize import word_tokenize
#from nltk.stem.snowball import SnowballStemmer
#from nltk.stem import WordNetLemmatizer

#from sklearn.feature_extraction import text
#from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
#extra = []
#extra = [e.lower() for e in extra]
#stop_words = text.ENGLISH_STOP_WORDS.union(extra)

# stemming resuelve terminaciones y derivaciones de palabras, pero sin una base morfológica
#stemmer    = SnowballStemmer("english") 
# resuelve en base a un diccionario morfológico de las palabras
#lemmatizer = WordNetLemmatizer()

#countv = CountVectorizer(ngram_range = (1,3), max_df=.20, min_df=0.01, stop_words=stop_words, lowercase=True)
#tfid   = TfidfTransformer(norm='l2')


def preprocessing_text(texto):
    words = word_tokenize(texto)
    return words

In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/arturo.torrestey/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
# inicializo el SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

def vader_sent_analyzer(texto):
    # calculo el compound ratio promedio de la sentences que se encuentran en texto
    # texto viene como una lista de palabras
    pos_val = []
    neg_val = []
    neu_val = []
    for sent in texto:
        ss = sid.polarity_scores(sent)
        pos_val.append(ss['pos'])
        neg_val.append(ss['neg'])
        neu_val.append(ss['neu'])
        
    return (np.mean(pos_val), np.mean(neg_val), np.mean(neu_val))

In [8]:
# Calculo de polaridad
        
# preproceso los datos de la columna texto y defino el vader compound ratio
tweets['tuplas'] = [vader_sent_analyzer(preprocessing_text(t)) for t in tweets.text]
tweets['pos'] = tweets['tuplas'].apply(lambda x: x[0])
tweets['neg'] = tweets['tuplas'].apply(lambda x: x[1])
tweets['neu'] = tweets['tuplas'].apply(lambda x: x[2])

# selecciono las columnas para trabajar
tweets = tweets.loc[:, ['date', 'pos', 'neg', 'neu']]

# agrupo tweets por fecha, promediando los scores de pos, neg y neutros
grouped_tweets = tweets.groupby('date').mean()

## Cálculo de Retornos

In [9]:
# armo el nombre del archivo a leer de los parametros
dir_name = '../data/original'
stock_name = 'NFLX_all_prices.txt'
index_name = 'NDX_all_prices.txt'

stockname  = os.path.join(dir_name, stock_name)
indexname  = os.path.join(dir_name, index_name)

In [10]:
# leo los archivos y le cambio los nombres a las columnas
stk = pd.read_csv(stockname, usecols=['Date', 'Adj Close'])
ind = pd.read_csv(indexname,usecols=['Date', 'Adj Close'])
stk.columns = ['date', 'nflx']
ind.columns = ['date', 'ndx']

In [11]:
# mergeo los df con los precios de la acción y el índice
data = pd.merge(stk, ind, on='date')

In [12]:
# paso la fecha a formato datetime
data['date'] = pd.to_datetime(data.date)

In [13]:
# armo un df con el total de los días en el rango de fechas de los archivos provistos
dias_totales = pd.DataFrame(pd.date_range(data.date.min(), data.date.max(), freq='D'), columns=['date'])

In [14]:
# mergeo para buscar los días sin valor en precios (fin de semana y feriados)
data = pd.merge(dias_totales, data, on='date', how='left')

In [15]:
# interpolo los nans con función cuadrática
data = data.interpolate(method='quadratic')

In [16]:
# calculo el retorno logarítmico
data['log_ret nflx'] = np.log(data.nflx / data.nflx.shift(1))
data['log_ret ndx']  = np.log(data.ndx / data.ndx.shift(1))
data['exc_ret']      = (data['log_ret nflx'] - data['log_ret ndx']) > 0
data['exc_ret%']     = data['log_ret nflx'] - data['log_ret ndx']

In [17]:
# armo el df para retornar con la fecha y el excess return de la acción eliminando
# la primera observación (sin retorno calculado)
grouped_returns = data.loc[1: , ['date', 'exc_ret', 'exc_ret%']]

In [18]:
grouped_returns.head()

,date,exc_ret,exc_ret%
1,2017-11-28,True,0.018367
2,2017-11-29,False,-0.039507
3,2017-11-30,False,-0.011582
4,2017-12-01,True,0.000300
5,2017-12-02,False,-0.000404


## Merger de los Tweets y Retornos

In [23]:
dataset = pd.merge(grouped_tweets, grouped_returns, on='date', how='left')

In [24]:
dataset = dataset.dropna(subset=['exc_ret%'])

In [25]:
dataset.head()

,date,pos,neg,neu,exc_ret,exc_ret%
5,2017-11-28,0.040453,0.010565,0.930767,True,0.018367
6,2017-11-29,0.037469,0.014695,0.925284,False,-0.039507
7,2017-11-30,0.051480,0.021632,0.913808,False,-0.011582
8,2017-12-01,0.053825,0.016789,0.911705,True,0.000300
9,2017-12-02,0.041612,0.019572,0.909338,False,-0.000404


In [26]:
dataset.to_csv('dataset_final.csv')